In [1]:
import time
import csv
import os
from pathlib import Path
import sys
sys.path.insert(0, '../')
import pycor.utils as utils
import pycor.parser as ps
import pycor.docresolver as dr
import pycor.langmodel as langmodel
import pycor.speechmodel as sm
import pycor.korutils as ku
import pycor.std.korstandard

stopwatch = utils.StopWatch()


docsize = 2

model_size = 700
outputpath = "../../output/"
model_path = outputpath + "model/" + str(model_size)

parser = ps.SentenceParser()
resolver = dr.DocResolver()

parser.wordmap.load(model_path)

resolver.setwordmap(parser.wordmap)

Init SentenceParser
Init DocResolver
Load  ../../output/model/700/heads.csv   소요시간: 0.452
Load  ../../output/model/700/tails.csv   소요시간: 0.008


In [2]:

def listfiles(path):
    result_arr = []
    filenames = os.listdir(path)
    for filename in filenames:
        full_filename = os.path.join(path, filename)
        result_arr.append(full_filename)
    return result_arr
    
files = []
files.extend( listfiles('../samples/news') )
files.extend( listfiles('../../data/news') )
files.extend( listfiles('../../data/blotter') )
files.extend( listfiles('../../data/zdnet') )
files.extend( listfiles('../../data/ciobiz') )
files.extend( listfiles('../../data/wiki') )
files.extend( listfiles('../../data/NP') )

print (len(files), "files")


3517 files


In [3]:

def printKwywords(keywords, words_array):
    print("===========")
    print(keywords)

def summarizeDocument(file, rate=0.1, count=3):
    if file.endswith(".txt") :
        print(file)
        sentence_array, words_array = parser.loadfile(file, verbose=False)
        keywords = resolver.extractKeywords(words_array, rate)
        if len(keywords) < 1:
            rate /= 2
            keywords = resolver.extractKeywords(words_array, rate)
        print("===========")
        print(keywords)
        print("------------")
        sentences = resolver.abstractDocument(keywords, words_array, count)
        dr.printSentences(sentences)
        return words_array


def summarizeDocument2(file, keywords, count=3):
    if file.endswith(".txt") :
        print(file)
        sentence_array, words_array = parser.loadfile(file, verbose=False)
        print("===========")
        print(keywords)
        print("------------")
        sentences = resolver.abstractDocument(keywords, words_array, count)
        dr.printSentences(sentences)
        return words_array


def extractKw(file, rate=0.1):
    if file.endswith(".txt") :
        print(file)
        sentence_array, words_array = parser.loadfile(file, verbose=False)
        keywords = resolver.extractKeywords(words_array,rate)
        printKwywords(keywords, words_array)


stopwatch.start()

for file in files[:docsize]:
#     extractKw(file, 0.05)
    summarizeDocument(file, 0.05, 3)

# extractKw("../data/NP/mp-all.txt",0.1)
# summarizeDocument("../data/NP/mp-all.txt",0.1)
# summarizeDocument("../data/wiki/%EB%8B%A8%EB%B0%B1%EC%A7%88.txt",0.1)
# summarizeDocument("samples/01_sampletext.txt")
# summarizeDocument("../data/wiki/%EC%A0%95%EB%B3%B4%20%EC%9D%B4%EB%A1%A0.txt")

print("Loading Texts: " , stopwatch.secmilli() , "(", stopwatch.millisecstr(), "ms.)")

../samples/news/new1_joins.txt
{전: 4, 지역: 6, 태풍예비특보: 4, 태풍: 6, 광주: 4, 22일: 6, 오후: 5, 오전: 2, 3시: 2, 약: 3, 부근: 3, 해상: 4, 23일: 2, 밤: 3, 일부: 2, 바람: 2, 영향권: 2, 해안: 2}
------------
Sentence Count: 3
제19호 태풍 [ 솔릭 [ SOULIK ] ] 이 북상함에 따라 광주와 전남 전 지역에 태풍예비특보가 내려졌다 . 
광주기상청은 오는 22일 오후를 기해 광주와 전남 22개 시군 전 지역흑산도홍도와 남해서부앞바다서해남부먼바다서해남부앞바다에 태풍예비특보를 발효한다고 21일 밝혔다 . 
태풍은 22일 오후 서귀포 해상을 지나 23일 오전 9시께 목포 남서쪽 약 120 해상에 이를 것으로 전망된다 . 
../samples/news/new1_hani.txt
{태풍: 9, 솔릭: 3, 제주: 2, 비상: 4, 제주도: 5, 휴: 3, 사용: 2, 하: 3, 피해: 2, 대비: 3, 태세: 2, 공무원: 2, 등: 3, 최대: 2, 바람: 2, 예상: 2, 이상: 2, 근무: 2, 활동: 2, 강화: 2, 상황: 2}
------------
Sentence Count: 3
제19호 태풍 솔릭이 제주도를 관통할 것으로 보여 제주도에 비상이 걸렸다 . 
제주도는 공무원들의 휴가 사용을 제한하는가 하면 태풍 피해를 최소화하기 위한 대비 태세에 들어갔다 . 
제주도는 태풍 솔릭의 북상에 따라 2223일 공무원의 연가 등 휴가 사용을 제한하고 휴가 중인 공무원 전원을 업무에 복귀시켜 피해 예방을 위한 대비 태세에 나섰다고 21일 밝혔다 . 
Loading Texts:  0s.054ms. ( 54 ms.)


In [4]:
words_array_list = []

for file in files[:4]:
    words_array = summarizeDocument(file, 0.05, 3)
    words_array_list.append(words_array)

print("======== Merge News ==========")
sentences = resolver.mergeDocuments(words_array_list, 0.3, 6)

dr.printSentences(sentences)


../samples/news/new1_joins.txt
{솔릭: 5, 전: 4, 지역: 6, 태풍예비특보: 4, 태풍: 6, 광주: 4, 22일: 6, 오후: 5, 오전: 2, 3시: 2, 약: 3, 부근: 3, 해상: 4, 23일: 2, 밤: 3, 일부: 2, 바람: 2, 영향권: 2, 해안: 2}
------------
Sentence Count: 3
제19호 태풍 [ 솔릭 [ SOULIK ] ] 이 북상함에 따라 광주와 전남 전 지역에 태풍예비특보가 내려졌다 . 
태풍 [ 솔릭 ] 은 이날 오후 3시 기준 일본 가고시마 남쪽 약 380 부근 해상에서 시속 23의 속도로 북서진하고 있다 . 
태풍은 22일 오후 서귀포 해상을 지나 23일 오전 9시께 목포 남서쪽 약 120 해상에 이를 것으로 전망된다 . 
../samples/news/new1_hani.txt
{태풍: 9, 솔릭: 3, 제주: 2, 비상: 4, 제주도: 5, 휴: 3, 사용: 2, 하: 3, 피해: 2, 대비: 3, 태세: 2, 공무원: 2, 등: 3, 최대: 2, 바람: 2, 예상: 2, 이상: 2, 근무: 2, 활동: 2, 강화: 2, 상황: 2}
------------
Sentence Count: 3
제19호 태풍 솔릭이 제주도를 관통할 것으로 보여 제주도에 비상이 걸렸다 . 
제주도는 공무원들의 휴가 사용을 제한하는가 하면 태풍 피해를 최소화하기 위한 대비 태세에 들어갔다 . 
제주도는 태풍 솔릭의 북상에 따라 2223일 공무원의 연가 등 휴가 사용을 제한하고 휴가 중인 공무원 전원을 업무에 복귀시켜 피해 예방을 위한 대비 태세에 나섰다고 21일 밝혔다 . 
../samples/news/new1_khan.txt
{정부: 2, 태풍: 8, 대비: 5, 가동: 2, 솔릭: 2, 대응: 2, 이날: 2, 오후: 2, 강풍: 3, 피해: 2, 행안부: 2, 댐: 2, 상황: 2, 사전: 2, 하: 2}
------------
Sentence Count: 3
제19호 태풍 [ 솔릭 ] 이 오는

In [6]:
words_array_list = []

for file in files[27:33]:
    words_array = summarizeDocument(file, 0.05, 3)
    words_array_list.append(words_array)

print("======== Merge News ==========")
sentences = resolver.mergeDocuments(words_array_list, 0.1, 6)

dr.printSentences(sentences)

../../data/news/khan-301.txt
{대통령: 8, 국가적: 2, 유지해: 2, 태풍: 7, 23일: 2, 오전: 2, 청와대: 3, 솔릭: 2, 대처: 3, 상황: 2, 전국: 2, 중앙정부: 2, 지자체: 3, 이번: 2, 바: 8, 말하: 3, 이날: 2, 학교: 2, 등: 6, 대비: 2, 지역: 3, 시간: 2, 안전: 4, 주기: 2, 강풍: 2, 피해: 4, 크레인: 2, 그런: 3, 해: 2, 주: 2, 지원: 2, 각별하: 2, 간: 2, 일정: 2}
------------
Sentence Count: 3
문재인 대통령은 23일 [ 중앙정부와 지자체를 포함한 모든 공직자들은 이번 태풍이 완전히 끝날 때까지 긴장의 끈을 놓지 말고 국가적 비상대비 태세를 유지해서 총력 대응해 주시기 바란다 ] 고 말했다 . 
문 대통령은 [ 일부 지역에서 이미 시작되었지만 지방교육청과 또 일선 학교를 포함한 전국의 모든 교육기관들이 임시휴교와 등하교 시간 조정 등 학생들의 안전을 위해서 가능한 모든 방법을 적극적으로 강구해 주기 바란다 ] 며 [ 아울러 민간 기업들도 직원들의 안전을 최우선적으로 고려해서 필요하다면 출퇴근 시간을 조정하는 등 능동적인 대처에 나설 수 있도록 그렇게 함께 노력을 해주시기 바란다 ] 고 말했다 . 
문 대통령은 [ 아까 통일부 장관 말씀이 계셨습니다만 지금 2차 남북 이산가족 상봉을 앞두고 있는데 이번 태풍이 그 지역 쪽으로 빠져나갈 것으로 그렇게 예상이 되고 있다 ] 며 [ 특히 연로하신 분들이 많기 때문에 또 그분들의 안전에 각별히 유의해 주시고 필요하다면 장소나 일정조정 등 가능한 모든 방안을 신속하게 검토를 해 주기 바란다 ] 고 말했다 . 
../../data/news/khan-315.txt
{개선: 3, 화장품: 4, 등: 2, 명의: 2, 함: 2, 암: 3, 정복: 2, 아카데미: 2, 이달: 2, 오후: 2, 이야기: 2, 교수: 2, 유천포창: 4, 피부: 2, 대상: 2, 효과: 2, 참가자: 2, 